# **<p style="text-align: center;">Aprendizaje por refuerzo - Ingeniería en Inteligencia Artificial</p>**
## **<p style="text-align: center;">Trabajo práctico Final - SAC</p>**

#### <p style="text-align: center;">Pettinari Fausto, Schuemer Ignacio, Torres Santiago </p>
#### <p style="text-align: center;">Profesores: Claudio Pose, Gabriel Torre, Nicolás Romero, Tomás Chimenti</p>

# **One DPoint Mass Reach Enviroment - *Entorno de Prueba***

In [1]:
import gymnasium as gym
from envs import OneDPointMassReachEnv

In [2]:
gym.register(
    id="gymnasium_env/OneDPointMassReachEnv-v0",
    entry_point=OneDPointMassReachEnv
)
env = gym.make("gymnasium_env/OneDPointMassReachEnv-v0")

#### **Logging & Debugging (Random Agent)**

In [3]:
from torch.utils.tensorboard import SummaryWriter
import sys
import os
sys.path.append(os.path.join('..'))
from src.random_agent import random_agent_loop

2025-11-23 21:24:29.011594: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 21:24:29.077751: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-23 21:24:30.864013: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
seed = 42
writer = SummaryWriter(f"../runs/OneDPointMassReachEnv")
num_episodes = 5000
# random_agent_loop(env, num_episodes, writer, seed)


## **SAC Agent**

In [5]:
from src.SAC import SAC
from src.hyperparameters import SACConfig

In [6]:
from dataclasses import replace
from src.alpha_scheduler import FixedAlpha
base = SACConfig()
experiment_cfg = SACConfig(
    sac=replace(base.sac, gamma=0.99, tau=0.01, alpha=FixedAlpha(0.01)),
    q_net=replace(base.q_net, hidden_sizes=[64, 64], hidden_layers_act="relu"),
    policy_net=replace(base.policy_net, log_std_min=-10, log_std_max=1, hidden_sizes=[64, 64], hidden_layers_act="relu"),
    buffer=replace(base.buffer, capacity=5000),
    train=replace(base.train, batch_size=256, device="cuda", warming_steps=1000),
    logger=replace(base.logger, env_name="OneDPointMassReachEnv", log_dir="../runs")
)

In [7]:
SAC_agent = SAC(env=env, config=experiment_cfg)
SAC_agent.run_training_loop(num_episodes=400, tqdm_disable=False, print_rewards=False)

100%|██████████| 400/400 [01:17<00:00,  5.15it/s]


{'total_episodes': 400,
 'best_avg_return': np.float64(0.8987000000000005),
 'final_avg_return': np.float64(0.8934000000000003)}